<a href="https://www.kaggle.com/code/klaidenx/aml-imaging?scriptVersionId=144456300" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<img src="https://www.sciencefacts.net/wp-content/uploads/2019/11/Types-of-White-Blood-Cells.jpg" alt="cell functions" style="width:8vw;"/>

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from glob import glob
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
general_path="../input/single-cell-morphological-dataset-of-leukocytes/blood_smear_images_for_aml_diagnosis_MOD/AML-Cytomorphology_LMU_MOD"
CLASSES=os.listdir(general_path)
CLASSES

In [ ]:
def read_data(path:str,cell_types:list):

    LEN=3
    if len(cell_types)==LEN:
        path_img=os.path.join(path,cell_types)
        single_type_img=[cv.imread(img) for img in glob(path_img+"/*.tiff")]
        return single_type_img
    else:
        cell_images={}
        for cell in cell_types:
            path_img=os.path.join(path,cell)
            cell_images[cell]=[cv.imread(img) for img in glob(path_img+"/*.tiff")]
        return cell_images

In [ ]:
basophils=read_data(general_path,CLASSES[0])

In [ ]:
full_imgs=read_data(general_path,CLASSES[:4])

In [ ]:
fig,axs=plt.subplots(2,5,figsize=(20,5))
axs=axs.flatten()
for i,ax in enumerate(axs):
    ax.imshow(basophils[i])

In [ ]:
class ImagePreprocess:
    
    def __init__(self,images):
        self.img=images
        self.gray_images=[cv.cvtColor(img,cv.COLOR_RGB2GRAY) for img in self.img]
        
    def calc_histogram(self):
        
        gray_hist=[]
        for imgs in self.gray_images:
            hist=cv.calcHist([imgs],[0],None,[256],[0,256])
            gray_hist.append(hist)
        return np.array(gray_hist).astype("float32")
    
    
    def show_plot(self,TYPE,LIMIT=5,image_cntr=None):
        
        
        fig,axs=plt.subplots(2,LIMIT,figsize=(25,5))
        if TYPE=="hist":
            axs=axs.flatten()
            for i,ax in enumerate(axs):
                ax.plot(self.calc_histogram()[i])
                ax.set_title("Gray Histogram")
            
        elif TYPE=="image":
            axs=axs.flatten()
            for i,ax in enumerate(axs):
                ax.imshow(image_cntr[i])
                ax.set_title("Cells Contours")
        fig.tight_layout(pad=.8)
        
    def draw_contours(self):
        
        cntr_list=[]
        draw_cntr=[]
        MIN_THRESHOLD=150
        MAX_THRESHOLD=254
        gray_imgs=self.gray_images
        mask_img=gray_imgs[0].copy()
        
        for img in gray_imgs:
            img=cv.GaussianBlur(img,(5,5),0)
            ret,threshold=cv.threshold(img,MIN_THRESHOLD,MAX_THRESHOLD,cv.THRESH_BINARY_INV)
            contours,hierarchy=cv.findContours(threshold,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
            cntr_list.append(contours)
        
        for i,img in enumerate(self.img):
            draw_cntr.append(cv.drawContours(img,cntr_list[i],-1,(0,255,0),2))
        return draw_cntr
    
    def data_augmentation(self):
        
        augmentation_options={
            "horizontal_flip":True,
            "vertical_flip":True,
            "zoom_range":(0.1,0.9),
            "rotation_range":(5,180),
            "rescale":1./255,
            "dtype":np.float32
        }
        
        return augmentation_options
            
basophils_preprocess=ImagePreprocess(basophils)

In [ ]:
basophils_preprocess.show_plot(TYPE="hist")

In [ ]:
bas_cntr=basophils_preprocess.draw_contours()
basophils_preprocess.show_plot(image_cntr=bas_cntr,TYPE="image")

In [ ]:
img_gen=ImageDataGenerator(
    basophils_preprocess.data_augmentation()
)
img_arr=np.array(basophils)
pics=img_gen.flow(img_arr,batch_size=2)

In [ ]:
plt.imshow(pics.next()[0].astype("uint8"))